In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import scipy as sp

In [2]:
train_data=pd.read_csv('train_data_li_ex.csv')#沥川清洗数据
test_data=pd.read_csv('test_data_li_ex.csv')

In [3]:
submit_data=pd.read_csv('submit_example.csv',header=None)
submit_data.columns=['ID','发电量']

In [4]:
#皮尔森相关系数0.70以上
list_att=['B_Voltage_X_A_Current', 'C_Voltage_X_A_Current',
       'A_voltage_X_A_Current', 'light_intensity_X_A_conversion_efficiency',
       'A_Current', 'B_Voltage_X_C_Current', 'A_voltage_X_C_Current',
       'C_Current', 'C_Voltage_X_C_Current',
       'light_intensity_X_C_conversion_efficiency', 'A_voltage_X_B_Current',
       'B_Current', 'C_Voltage_X_B_Current', 'B_Voltage_X_B_Current',
       'light_intensity_X_B_conversion_efficiency', 'A_Current_X_C_Current',
       'A_Current_X_B_Current', 'B_Current_X_C_Current',
       'light_intensity_X_A_Current', 'light_intensity_X_B_Current',
       'light_intensity_X_C_Current', 'light_intensity_X_A_voltage',
       'light_intensity_X_B_Voltage', 'light_intensity_X_C_Voltage',
       'light_intensity', 'A_Current_X_wind_direction',
       'C_Current_X_wind_direction', 'B_Current_X_wind_direction',
       'light_intensity_X_wind_direction', 'A_Current_X_Wind_Speed',
       'B_Current_X_Wind_Speed', 'C_Current_X_Wind_Speed',
       'plate_temperature_X_C_Current', 'plate_temperature_X_A_Current',
       'plate_temperature_X_B_Current', 'plate_temperature_X_light_intensity',
       'light_intensity_X_Wind_Speed']

In [26]:
#train_data_li_ex
list_att=['B_Voltage_X_A_Current', 'C_Voltage_X_A_Current',
       'A_voltage_X_A_Current', 'light_intensity_X_A_conversion_efficiency',
       'A_Current', 'B_Voltage_X_C_Current', 'A_voltage_X_C_Current',
       'C_Current', 'C_Voltage_X_C_Current',
       'light_intensity_X_C_conversion_efficiency', 'A_voltage_X_B_Current',
       'B_Current', 'C_Voltage_X_B_Current', 'B_Voltage_X_B_Current',
       'light_intensity_X_B_conversion_efficiency', 'A_Current_X_C_Current',
       'A_Current_X_B_Current', 'B_Current_X_C_Current',
       'light_intensity_X_A_Current', 'light_intensity_X_B_Current',
       'light_intensity_X_C_Current', 'light_intensity_X_A_voltage',
       'light_intensity_X_B_Voltage', 'light_intensity_X_C_Voltage',
       'light_intensity', 'A_Current_X_wind_direction',
       'C_Current_X_wind_direction', 'B_Current_X_wind_direction',
       'light_intensity_X_wind_direction', 'A_Current_X_Wind_Speed',
       'B_Current_X_Wind_Speed', 'C_Current_X_Wind_Speed',
       'plate_temperature_X_C_Current', 'plate_temperature_X_A_Current',
       'plate_temperature_X_B_Current', 'plate_temperature_X_light_intensity',
       'light_intensity_X_Wind_Speed', 'A_conversion_efficiency_X_B_Current',
       'C_conversion_efficiency_X_B_Current',
       'B_conversion_efficiency_X_C_Current', 'plate_temperature_X_A_voltage',
       'plate_temperature_X_C_Voltage', 'plate_temperature_X_B_Voltage',
       'plate_temperature', 'A_conversion_efficiency_X_C_Current',
       'A_conversion_efficiency_X_A_Current',
       'C_conversion_efficiency_X_A_Current',
       'plate_temperature_X_wind_direction',
       'B_conversion_efficiency_X_A_Current',
       'C_conversion_efficiency_X_C_Current',
       'B_conversion_efficiency_X_B_Current',
       'plate_temperature_X_B_conversion_efficiency', 'C_Current_D_A_voltage',
       'light_intensity_D_A_conversion_efficiency',
       'plate_temperature_X_A_conversion_efficiency',
       'plate_temperature_X_Wind_Speed',
       'plate_temperature_X_C_conversion_efficiency',
       'plate_temperature_D_A_Current', 'plate_temperature_D_Wind_Speed',
       'plate_temperature_D_C_Current', 'site_temperature_D_A_Current',
       'Wind_Speed_X_wind_direction', 'wind_direction',
       ]

In [27]:
train_x=train_data[list_att][:8500]
train_y=train_data['power_generation'][:8500]

In [28]:
val_x=train_data[list_att][8500:]
val_y=train_data['power_generation'][8500:]

In [29]:
test_x=test_data[list_att]

## XGboost

In [9]:
trainset = xgb.DMatrix(train_x, label=train_y)
valset = xgb.DMatrix(val_x)
testset=xgb.DMatrix(test_x)
params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L3 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree':1 , # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model = xgb.train(params, trainset, num_boost_round=2000)
predict_xgb = model.predict(valset)
score=1/(1+np.sqrt(np.average(np.square(predict_xgb-val_y.values))))
print(score)

0.853559283957


In [10]:
predict_xgb_sub = model.predict(testset)

In [11]:
submit_data['发电量']=predict_xgb_sub

In [12]:
submit_data.to_csv('submit/result_0805_XGBoost_1.csv',index=False, header=None) 

In [ ]:
0.8535

## GBDT

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [40]:
est = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.01,
                                   max_depth=6, max_features='sqrt',
                                   min_samples_leaf=1, min_samples_split=10,
                                   loss='huber',random_state=42).fit(train_x, train_y)
gbdt_pred = est.predict(val_x)
score=1/(1+np.sqrt(np.average(np.square(gbdt_pred-val_y.values))))
print(score)

0.881421232838


In [ ]:
est = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.01,
                                   max_depth=5, max_features='sqrt',
                                   min_samples_leaf=1, min_samples_split=2,
                                   loss='huber',random_state=42).fit(train_x, train_y)
gbdt_pred = est.predict(val_x)
score=1/(1+np.sqrt(np.average(np.square(gbdt_pred-val_y.values))))
print(score)

历次线下成绩
0.86890016021
0.864988414897
0.865329251732
0.871572079478
0.8736 min_samples_split=15
0.872679296451
0.877096033043/0.83588
0.8820/0.8838
0.88292312227/
0.8555
0.879806053751
0.8766
0.881082405649
0.882457554823

In [16]:
est = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.01,
                                   max_depth=6, max_features='sqrt',
                                   min_samples_leaf=13, min_samples_split=15,
                                   loss='huber').fit(train_data[list_att], train_data['power_generation'])
gbdt_pred = est.predict(test_data[list_att])

In [17]:
submit_data['发电量']=gbdt_pred
submit_data.to_csv('submit/result_180805_gbdt_1.csv',index=False, header=None) 

## 线性回归

In [33]:
from sklearn import cross_validation
from sklearn import linear_model

/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [34]:
from sklearn.metrics import mean_squared_error

In [35]:
# watch scores
clf_ = linear_model.LinearRegression()
X_ = train_data[list_att].as_matrix()
y_ = train_data['power_generation'].as_matrix()
X_1=train_x.as_matrix()
y_1=train_y.as_matrix()
clf_.fit(X_1,y_1)
X_2=val_x.as_matrix()
y_pred=clf_.predict(X_2)
print(1/(1+np.sqrt(np.average(np.square(y_pred-val_y.values)))))

/anaconda/envs/py36/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

0.84374382052


## kfold 

In [53]:
from sklearn.cross_validation import cross_val_score

In [ ]:
clf_ = linear_model.LinearRegression()
X_ = train_data[list_att].as_matrix()
y_ = train_data['power_generation'].as_matrix()
loss = -cross_val_score(clf_, X_, y_, cv=10, scoring='mean_squared_error')
1/(1+np.sqrt(loss))

## 深度神经

In [64]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [71]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
model = Sequential([
    Dense(8192, input_dim=train_x.shape[1]),
    Activation('relu'),
    Dense(4096),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(1),
])
model.compile(loss='mse', optimizer='sgd')
model.fit(train_x.as_matrix(), train_y.as_matrix(), batch_size=1024, epochs=20, verbose=1,validation_data=(val_x.as_matrix(),val_y.as_matrix()), shuffle=True)

In [ ]:
y_pred=model.predict(val_x.as_matrix())

## LightGBM

In [13]:
from lightgbm import LGBMRegressor

In [ ]:
model=LGBMRegressor(boosting_type='gbdt', num_leaves=63, max_depth=5, learning_rate=0.1, n_estimators=1000, subsample_for_bin=200000, objective='regression', class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.1, random_state=42, n_jobs=-1, silent=False, importance_type='split')

model.fit(train_x, train_y,eval_set=(val_x,val_y))
lgb_pred = model.predict(val_x)
print(1/(1+np.sqrt(np.average(np.square(lgb_pred-val_y.values)))))


In [14]:
lgb_train = lgb.Dataset(train_x, train_y)

params = {
'learning_rate': 0.1,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 1,
'num_leaves': 30,
'min_data': 20,
'min_hessian': 1,
'n_estimators':100,
'verbose': 1,}

model = lgb.train(params, lgb_train, 600)
lgb_pred = model.predict(val_x)
print(1/(1+np.sqrt(np.average(np.square(lgb_pred-val_y.values)))))

/anaconda/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.857011954832


In [25]:
submit_data['发电量']=lgb_pred
submit_data.to_csv('submit/result_180805_lgb_1.csv',index=False, header=None) 

In [ ]:
(不考虑功率情况)0.855112552951/
(考虑功率的情况) 0.867786153325/0.833
0.850856164589/0.8218

In [31]:
merge1=pd.read_csv('submit/result_180731_gbdt_3.csv',header=None)
merge1.head()

,0,1
0,1,0.379571
1,9,1.340937
2,13,2.181017
3,17,3.390921
4,18,3.631887


In [32]:
merge2=pd.read_csv('submit/result_180805_gbdt_1.csv',header=None)
merge2.head()

,0,1
0,1,0.379972
1,9,1.239851
2,13,2.208346
3,17,3.400142
4,18,3.667668


In [35]:
def merge_fun(merge1,merge2):
    merge1[1]=0.5*merge1[1]+0.5*merge2[1]
    return merge1

In [36]:
merge_result=merge_fun(merge1,merge2)
merge_result.to_csv('submit/result_180805_merge1.csv',index=False, header=None) 

In [37]:
s1=pd.read_csv('submit/result_180805_merge1.csv', header=None)
s1.head()

,0,1
0,1,0.379772
1,9,1.290394
2,13,2.194681
3,17,3.395532
4,18,3.649777


## 贝叶斯xgboost

In [68]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from bayes_opt import BayesianOptimization
from tqdm import tqdm


In [ ]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)


    cv_result = xgb.cv(params, xgtrain, num_boost_round=num_rounds, nfold=5,
             seed=random_state,
             callbacks=[xgb.callback.early_stop(50)])

    return -cv_result['test-mae-mean'].values[-1]

def prepare_data():
    
    y = train_data['power_generation']

    X = train_data[list_att]
    xgtrain = xgb.DMatrix(X, label=y)

    return xgtrain


if __name__ == '__main__':
    xgtrain = prepare_data()

    num_rounds = 3000
    random_state = 2016
    num_iter = 25
    init_points = 5
    params = {
        'eta': 0.1,
        'silent': 1,
        'eval_metric': 'mae',
        'verbose_eval': True,
        'seed': random_state
    }

    xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                                'colsample_bytree': (0.1, 1),
                                                'max_depth': (5, 15),
                                                'subsample': (0.5, 1),
                                                'gamma': (0, 10),
                                                'alpha': (0, 10),
                                                })

    xgbBO.maximize(init_points=init_points, n_iter=num_iter)
